In [275]:
import pandas as pd
import numpy as np
import math
import random

In [276]:
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random.uniform(-(2/n_inputs),2/n_inputs) for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random.uniform(-(2/n_hidden),2/n_hidden) for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [277]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    activation += weights[-1]
    return activation

In [278]:
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [279]:
def transfer_derivative(output):
        return output * (1.0 - output)

In [280]:
def forward_propagate(network, row):
        inputs = row
        for layer in network:
                new_inputs = []
                for neuron in layer:
                        activation = activate(neuron['weights'], inputs)
                        neuron['output'] = transfer(activation)
                        new_inputs.append(neuron['output'])
                inputs = new_inputs
        return inputs

In [281]:
def backward_propagate_error(network, expected):
    out_error = 0.0
    oLayer = network[-1]
    for i in range(len(network[-1])):
        neuron = oLayer[i]
        neuron['delta'] = (expected - neuron['output']) * transfer_derivative(neuron['output'])
        out_error = neuron['delta']
    for j in reversed(range(len(network[:-1]))):
        layer = network[j]
    for i, neuron in enumerate(layer):
        neuron['delta'] = (network[-1][0]['weights'][i] * out_error) * transfer_derivative(neuron['output'])

In [282]:
def update_weights(network, inputs, l_rate):
    for k in range(len(network)):
        layer = network[k]
        for j in range(len(layer)):
            neuron = layer[j]
            for i, input in enumerate(inputs):
                neuron['weights'][i] += l_rate * neuron['delta'] * input
            neuron['weights'][-1] += l_rate * neuron['delta']

In [283]:
#Update weights with momentum
def update_weightsM(network, inputs, l_rate):
    for k in range(len(network)):
        layer = network[k]
        for j in range(len(layer)):
            neuron = layer[j]
            prev_weight = neuron['weights'][:]
            for i, input in enumerate(inputs):
                if 'prev_weights' in neuron:
                    neuron['weights'][i] += l_rate * neuron['delta'] * input + (0.9 * (neuron['weights'][i] - neuron['prev_weights'][i]))
                else:
                    neuron['weights'][i] += l_rate * neuron['delta'] * input
            if 'prev_weights' in neuron:
                neuron['weights'][-1] += l_rate * neuron['delta'] + (0.9 * (neuron['weights'][-1] - neuron['prev_weights'][-1]))
            else:
                neuron['weights'][-1] += l_rate * neuron['delta']
            neuron['prev_weights'] = prev_weight

In [284]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs[0]

In [285]:
def train_network(network, train, expected, l_rate, n_epoch):
    for epoch in range(n_epoch):
        sum_error = 0
        for i, row in enumerate(train):
            outputs = forward_propagate(network, row)
            sum_error += sum([(expected[i]-outputs[0])**2])
            backward_propagate_error(network, expected[i])
            update_weightsM(network, row, l_rate)
        RMSE = math.sqrt(sum_error/len(train))
        print('epoch=%d, lrate=%.3f, error=%.3f , rmse=%.6f' % (epoch, l_rate, sum_error, RMSE))

In [144]:
#Training the network with time based learning rate
def train_network(network, train, expected, l_rate, n_epoch):
    decay = l_rate / n_epoch
    for epoch in range(n_epoch):
        sum_error = 0
        l_rate = l_rate * 1/(1 + decay * epoch)
        for i, row in enumerate(train):
            outputs = forward_propagate(network, row)
            sum_error += sum([(expected[i]-outputs[0])**2])
            backward_propagate_error(network, expected[i])
            update_weightsM(network, row, l_rate)
        RMSE = math.sqrt(sum_error/len(train))
        print('epoch=%d, lrate=%.3f, error=%.3f , rmse=%.6f' % (epoch, l_rate, sum_error, RMSE))

In [286]:
# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

In [287]:
# Rescale dataset columns to the range 0-1
def standardize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (0.8 * (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])) + 0.1

In [288]:
def destandardize_dataset(dataset, minmax):
     for row in dataset:
        for i in range(len(row)):
            row[i] = (((row[i] - 0.1) / 0.8) * (minmax[i][1] - minmax[i][0]) + minmax[i][0])

In [289]:
#Read csv file, containing the data
df = pd.read_csv('StudentData.csv')
#Get the dataset values in an array
dataset = df.values
#Return an array containing the minimum and maximum values for each column in the dataset
minmax = dataset_minmax(dataset)
#Standardize the dataset
standardize_dataset(dataset, minmax)

In [290]:
split = math.ceil(0.6 * len(dataset))
split1 = math.ceil(0.8 * len(dataset))

#Validation set
valid = dataset[split1:,:]
validX, validY = valid[:, :-1], valid[:, -1]

#Test set
test = dataset[split:split1,:]
testX, testY = test[:, :-1], test[:, -1]

#Train set
train =  dataset[:split,:]
trainX, trainY = train[:, :-1], train[:, -1]

In [291]:
network = initialize_network(trainX.shape[1], 7, 1)

In [252]:
l_rate = 0.1
n_epoch = 1000
train_network(network, trainX, trainY, l_rate, n_epoch)

epoch=0, lrate=0.100, error=15.518 , rmse=0.133323
epoch=1, lrate=0.100, error=2.675 , rmse=0.055357
epoch=2, lrate=0.100, error=2.447 , rmse=0.052946
epoch=3, lrate=0.100, error=2.332 , rmse=0.051683
epoch=4, lrate=0.100, error=2.254 , rmse=0.050810
epoch=5, lrate=0.100, error=2.196 , rmse=0.050155
epoch=6, lrate=0.100, error=2.152 , rmse=0.049651
epoch=7, lrate=0.100, error=2.118 , rmse=0.049259
epoch=8, lrate=0.100, error=2.092 , rmse=0.048954
epoch=9, lrate=0.100, error=2.072 , rmse=0.048715
epoch=10, lrate=0.100, error=2.056 , rmse=0.048526
epoch=11, lrate=0.100, error=2.043 , rmse=0.048375
epoch=12, lrate=0.100, error=2.033 , rmse=0.048254
epoch=13, lrate=0.100, error=2.024 , rmse=0.048155
epoch=14, lrate=0.100, error=2.018 , rmse=0.048073
epoch=15, lrate=0.100, error=2.012 , rmse=0.048004
epoch=16, lrate=0.100, error=2.007 , rmse=0.047945
epoch=17, lrate=0.100, error=2.002 , rmse=0.047893
epoch=18, lrate=0.100, error=1.999 , rmse=0.047847
epoch=19, lrate=0.100, error=1.995 , rms

epoch=161, lrate=0.100, error=1.835 , rmse=0.045842
epoch=162, lrate=0.100, error=1.834 , rmse=0.045838
epoch=163, lrate=0.100, error=1.834 , rmse=0.045833
epoch=164, lrate=0.100, error=1.834 , rmse=0.045829
epoch=165, lrate=0.100, error=1.833 , rmse=0.045826
epoch=166, lrate=0.100, error=1.833 , rmse=0.045822
epoch=167, lrate=0.100, error=1.833 , rmse=0.045818
epoch=168, lrate=0.100, error=1.832 , rmse=0.045814
epoch=169, lrate=0.100, error=1.832 , rmse=0.045810
epoch=170, lrate=0.100, error=1.832 , rmse=0.045807
epoch=171, lrate=0.100, error=1.831 , rmse=0.045803
epoch=172, lrate=0.100, error=1.831 , rmse=0.045799
epoch=173, lrate=0.100, error=1.831 , rmse=0.045796
epoch=174, lrate=0.100, error=1.831 , rmse=0.045792
epoch=175, lrate=0.100, error=1.830 , rmse=0.045789
epoch=176, lrate=0.100, error=1.830 , rmse=0.045785
epoch=177, lrate=0.100, error=1.830 , rmse=0.045782
epoch=178, lrate=0.100, error=1.830 , rmse=0.045779
epoch=179, lrate=0.100, error=1.829 , rmse=0.045776
epoch=180, l

epoch=319, lrate=0.100, error=1.804 , rmse=0.045457
epoch=320, lrate=0.100, error=1.804 , rmse=0.045455
epoch=321, lrate=0.100, error=1.804 , rmse=0.045452
epoch=322, lrate=0.100, error=1.803 , rmse=0.045450
epoch=323, lrate=0.100, error=1.803 , rmse=0.045447
epoch=324, lrate=0.100, error=1.803 , rmse=0.045444
epoch=325, lrate=0.100, error=1.803 , rmse=0.045442
epoch=326, lrate=0.100, error=1.802 , rmse=0.045439
epoch=327, lrate=0.100, error=1.802 , rmse=0.045436
epoch=328, lrate=0.100, error=1.802 , rmse=0.045433
epoch=329, lrate=0.100, error=1.802 , rmse=0.045431
epoch=330, lrate=0.100, error=1.802 , rmse=0.045428
epoch=331, lrate=0.100, error=1.801 , rmse=0.045425
epoch=332, lrate=0.100, error=1.801 , rmse=0.045422
epoch=333, lrate=0.100, error=1.801 , rmse=0.045420
epoch=334, lrate=0.100, error=1.801 , rmse=0.045417
epoch=335, lrate=0.100, error=1.801 , rmse=0.045414
epoch=336, lrate=0.100, error=1.800 , rmse=0.045411
epoch=337, lrate=0.100, error=1.800 , rmse=0.045408
epoch=338, l

epoch=478, lrate=0.100, error=1.754 , rmse=0.044828
epoch=479, lrate=0.100, error=1.754 , rmse=0.044822
epoch=480, lrate=0.100, error=1.753 , rmse=0.044815
epoch=481, lrate=0.100, error=1.753 , rmse=0.044809
epoch=482, lrate=0.100, error=1.752 , rmse=0.044802
epoch=483, lrate=0.100, error=1.752 , rmse=0.044796
epoch=484, lrate=0.100, error=1.751 , rmse=0.044789
epoch=485, lrate=0.100, error=1.751 , rmse=0.044783
epoch=486, lrate=0.100, error=1.750 , rmse=0.044776
epoch=487, lrate=0.100, error=1.750 , rmse=0.044769
epoch=488, lrate=0.100, error=1.749 , rmse=0.044762
epoch=489, lrate=0.100, error=1.749 , rmse=0.044755
epoch=490, lrate=0.100, error=1.748 , rmse=0.044748
epoch=491, lrate=0.100, error=1.748 , rmse=0.044741
epoch=492, lrate=0.100, error=1.747 , rmse=0.044734
epoch=493, lrate=0.100, error=1.746 , rmse=0.044727
epoch=494, lrate=0.100, error=1.746 , rmse=0.044720
epoch=495, lrate=0.100, error=1.745 , rmse=0.044712
epoch=496, lrate=0.100, error=1.745 , rmse=0.044705
epoch=497, l

epoch=636, lrate=0.100, error=1.631 , rmse=0.043221
epoch=637, lrate=0.100, error=1.630 , rmse=0.043214
epoch=638, lrate=0.100, error=1.630 , rmse=0.043207
epoch=639, lrate=0.100, error=1.629 , rmse=0.043200
epoch=640, lrate=0.100, error=1.629 , rmse=0.043193
epoch=641, lrate=0.100, error=1.628 , rmse=0.043186
epoch=642, lrate=0.100, error=1.628 , rmse=0.043179
epoch=643, lrate=0.100, error=1.627 , rmse=0.043172
epoch=644, lrate=0.100, error=1.627 , rmse=0.043165
epoch=645, lrate=0.100, error=1.626 , rmse=0.043159
epoch=646, lrate=0.100, error=1.626 , rmse=0.043152
epoch=647, lrate=0.100, error=1.625 , rmse=0.043146
epoch=648, lrate=0.100, error=1.625 , rmse=0.043139
epoch=649, lrate=0.100, error=1.624 , rmse=0.043133
epoch=650, lrate=0.100, error=1.624 , rmse=0.043127
epoch=651, lrate=0.100, error=1.623 , rmse=0.043121
epoch=652, lrate=0.100, error=1.623 , rmse=0.043115
epoch=653, lrate=0.100, error=1.622 , rmse=0.043109
epoch=654, lrate=0.100, error=1.622 , rmse=0.043103
epoch=655, l

epoch=794, lrate=0.100, error=1.588 , rmse=0.042645
epoch=795, lrate=0.100, error=1.588 , rmse=0.042643
epoch=796, lrate=0.100, error=1.587 , rmse=0.042642
epoch=797, lrate=0.100, error=1.587 , rmse=0.042640
epoch=798, lrate=0.100, error=1.587 , rmse=0.042638
epoch=799, lrate=0.100, error=1.587 , rmse=0.042636
epoch=800, lrate=0.100, error=1.587 , rmse=0.042634
epoch=801, lrate=0.100, error=1.587 , rmse=0.042632
epoch=802, lrate=0.100, error=1.587 , rmse=0.042630
epoch=803, lrate=0.100, error=1.586 , rmse=0.042628
epoch=804, lrate=0.100, error=1.586 , rmse=0.042626
epoch=805, lrate=0.100, error=1.586 , rmse=0.042624
epoch=806, lrate=0.100, error=1.586 , rmse=0.042622
epoch=807, lrate=0.100, error=1.586 , rmse=0.042620
epoch=808, lrate=0.100, error=1.586 , rmse=0.042618
epoch=809, lrate=0.100, error=1.586 , rmse=0.042617
epoch=810, lrate=0.100, error=1.585 , rmse=0.042615
epoch=811, lrate=0.100, error=1.585 , rmse=0.042613
epoch=812, lrate=0.100, error=1.585 , rmse=0.042611
epoch=813, l

epoch=953, lrate=0.100, error=1.570 , rmse=0.042407
epoch=954, lrate=0.100, error=1.570 , rmse=0.042405
epoch=955, lrate=0.100, error=1.570 , rmse=0.042404
epoch=956, lrate=0.100, error=1.570 , rmse=0.042403
epoch=957, lrate=0.100, error=1.570 , rmse=0.042402
epoch=958, lrate=0.100, error=1.570 , rmse=0.042401
epoch=959, lrate=0.100, error=1.569 , rmse=0.042400
epoch=960, lrate=0.100, error=1.569 , rmse=0.042399
epoch=961, lrate=0.100, error=1.569 , rmse=0.042398
epoch=962, lrate=0.100, error=1.569 , rmse=0.042397
epoch=963, lrate=0.100, error=1.569 , rmse=0.042395
epoch=964, lrate=0.100, error=1.569 , rmse=0.042394
epoch=965, lrate=0.100, error=1.569 , rmse=0.042393
epoch=966, lrate=0.100, error=1.569 , rmse=0.042392
epoch=967, lrate=0.100, error=1.569 , rmse=0.042391
epoch=968, lrate=0.100, error=1.569 , rmse=0.042390
epoch=969, lrate=0.100, error=1.569 , rmse=0.042389
epoch=970, lrate=0.100, error=1.569 , rmse=0.042388
epoch=971, lrate=0.100, error=1.568 , rmse=0.042387
epoch=972, l

In [257]:
prediction = []
sum_error=0

for i, row in enumerate(testX):
    prediction.append(predict(network, row))
    print(prediction[i])
    
for i, row in enumerate(testY):
    sum_error += sum([(row-prediction[i])**2])
    RMSE = math.sqrt(sum_error/len(testX))
print('error=%.3f , rmse=%.6f' % (sum_error, RMSE))

0.5156168979183888
0.5529370912451285
0.7200262798868668
0.5300409538919317
0.5360387824306398
0.17192405738933575
0.31092509120316414
0.1846160298375338
0.17379953233430137
0.20005147568499254
0.1252848919339732
0.17981199324646963
0.1782166188242378
0.34275297713305897
0.12972681515076478
0.16097008463512413
0.13833738674032867
0.48242581637268506
0.5582001959904351
0.5913371420803079
0.26159926912631876
0.7462113923303413
0.8420440269159141
0.4832766100039577
0.3498408158587875
0.6451337444926216
0.12109203504852457
0.1847868424954507
0.10605801126454752
0.2607621107119054
0.16496513330060997
0.3137408156600893
0.32940915157771705
0.8539604516238367
0.4324857590926937
0.4757510358087579
0.18900234943645416
0.26906603623620423
0.27703173100013817
0.24229648603017281
0.22903492980220863
0.7412820347543227
0.7879278370217812
0.4907916175198433
0.5592996789186769
0.1649832825453146
0.18161450869408013
0.16907646174164742
0.3106108126465903
0.3750493791043742
0.6411276356158664
0.1761700